In [ ]:
%pip install pandas numpy matplotlib plotly seaborn openpyxl

In [ ]:
import os
import pandas as pd

# import numpy as np
# import matplotlib.pyplot as plt
# import plotly as py
# import plotly.graph_objs as go
# import plotly.express as px
# import seaborn as sns


def get_egresos_csv_files(root_path):
    csv_files = [
        os.path.join(root_path, file)
        for file in os.listdir(root_path)
        if file.endswith(".csv")
    ]
    return sorted(csv_files)


egresos_root_path = "../../datasets/egresos-hospitalarios"
egresos_folders = sorted(os.listdir(egresos_root_path))
egresos_csvs = get_egresos_csv_files(egresos_root_path)

In [ ]:
egresos_df = pd.DataFrame()

for csv_file in egresos_csvs:
    egresos = pd.read_csv(
        csv_file,
        delimiter=";",
        converters={
            "CONDICION_EGRESO": lambda v: True if v == "2" else False,
            "INTERV_Q": lambda v: True if v == "1" else False,
            "PROCED": lambda v: True if v == "1" else False,
        },
    )
    egresos.columns = egresos.columns.str.lower()
    egresos.rename(
        columns={
            "ano_egreso": "año",
            "diag1": "diagnostico_principal",
            "diag2": "causa_externa",
            "dias_estada": "dias_hospitalizacion",
            "interv_q": "intervencion_quirurgica",
            "proced": "procedimiento",
            "condicion_egreso": "egreso_muerte",
        },
        inplace=True,
    )

    if "glosa_interv_q_ppal" in egresos.columns:
        egresos.insert(
            egresos.columns.get_loc("egreso_muerte") + 1,
            "intervencion_quirurgica",
            None,
        )
        egresos["intervencion_quirurgica"] = egresos["glosa_interv_q_ppal"].notna()

    if "glosa_proced_ppal" in egresos.columns:
        egresos.insert(
            egresos.columns.get_loc("intervencion_quirurgica") + 1,
            "procedimiento",
            None,
        )
        egresos["procedimiento"] = egresos["glosa_proced_ppal"].notna()

    if "intervencion_quirurgica" not in egresos.columns:
        egresos.insert(
            egresos.columns.get_loc("egreso_muerte") + 1,
            "intervencion_quirurgica",
            None,
        )

    if "procedimiento" not in egresos.columns:
        egresos.insert(
            egresos.columns.get_loc("intervencion_quirurgica") + 1,
            "procedimiento",
            None,
        )

    egresos.drop(
        columns=[
            "pertenencia_establecimiento_salud",
            "pertenencia_establecimiento_salu",
            "prevision",
            "glosa_prevision",
            "region_residencia",
            "glosa_region_residencia",
            "sexo",
            "glosa_interv_q_ppal",
            "glosa_proced_ppal",
        ],
        inplace=True,
        errors="ignore",
    )
    egresos["procedimiento"] = egresos["procedimiento"].astype(bool)
    egresos["intervencion_quirurgica"] = egresos["intervencion_quirurgica"].astype(bool)
    egresos_df = pd.concat([egresos_df, egresos], ignore_index=True)

print(egresos_df["egreso_muerte"].value_counts())
print(egresos_df["intervencion_quirurgica"].value_counts())
print(egresos_df["procedimiento"].value_counts())
print(egresos_df.info())

In [ ]:
egresos_df.head()

In [ ]:
egresos_df[["diagnostico_principal"]].value_counts(dropna=False)

In [ ]:
diagnosticos = pd.read_excel(
    "../../datasets/egresos-hospitalarios/diccionario.xlsx",
    sheet_name="codigo CIE-10",
    skiprows=8,
)
diagnosticos.columns = diagnosticos.columns.str.lower().str.replace(" ", "_")
diagnosticos.head()

In [ ]:
egresos_diag_df = pd.merge(
    egresos_df,
    diagnosticos.add_prefix("diagnostico_"),
    left_on="diagnostico_principal",
    right_on="diagnostico_codigo_subcategoria",
)
print(egresos_diag_df.info())
egresos_diag_df.head()

In [ ]:
egresos_diag_df[["causa_externa"]].isna().value_counts(dropna=False)

In [ ]:
egresos_diag_causa_df = pd.merge(
    egresos_diag_df,
    diagnosticos.add_prefix("causa_externa_"),
    left_on="causa_externa",
    right_on="causa_externa_codigo_subcategoria",
    how="left",
)

print(egresos_diag_causa_df.info())
print(egresos_diag_causa_df.head())

egresos_diag_causa_df["diagnostico_principal"] = egresos_diag_causa_df[
    "diagnostico_glosa_subcategoria"
]
egresos_diag_causa_df["causa_externa"] = egresos_diag_causa_df[
    "causa_externa_glosa_subcategoria"
]

In [ ]:
print("isna?", egresos_diag_causa_df[["diagnostico_principal"]].isna().value_counts())
print("isna?", egresos_diag_causa_df[["causa_externa"]].isna().value_counts())

In [ ]:
egresos_diag_causa_df.drop(
    columns=[
        "diagnostico_codigo_subcategoria",
        "diagnostico_glosa_subcategoria",
        "diagnostico_capitulo",
        "diagnostico_codigo_grupo_",
        "diagnostico_codigo_categoria",
        "causa_externa_codigo_subcategoria",
        "causa_externa_glosa_subcategoria",
        "causa_externa_capitulo",
        "causa_externa_codigo_grupo_",
        "causa_externa_codigo_categoria",
    ],
    inplace=True,
    errors="ignore",
)

In [ ]:
egresos_diag_causa_df.info()

In [ ]:
egresos_diag_causa_df[["comuna_residencia"]].isna().value_counts()

In [ ]:
%run ../load_comunas_servicios.py "../../datasets/comunas_por_zona.csv"

comunas_servicios_df.rename(columns={"zona_id": "capitulo", "zona": "nombre_capitulo"}, inplace=True)
print(comunas_servicios_df.info())
comunas_servicios_df.head()

In [ ]:
comunas_servicios_df[comunas_servicios_df["comuna"] == "Chaitén"]

In [ ]:
egresos_diag_causa_df = pd.merge(
    egresos_diag_causa_df,
    comunas_servicios_df[["capitulo", "nombre_capitulo", "comuna_id"]],  # type: ignore
    left_on="comuna_residencia",
    right_on="comuna_id",
    how="left",
)

In [ ]:
print(egresos_diag_causa_df[["capitulo"]].isna().value_counts())
egresos_diag_causa_df[egresos_diag_causa_df["capitulo"].isna()]

In [ ]:
print(
    egresos_diag_causa_df[egresos_diag_causa_df["capitulo"].isna()][
        "comuna_residencia"
    ].unique()
)
print(
    egresos_diag_causa_df[egresos_diag_causa_df["capitulo"].isna()][
        "glosa_comuna_residencia"
    ].unique()
)

In [ ]:
missing_capitulos = egresos_diag_causa_df[egresos_diag_causa_df["capitulo"].isna()][
    ["comuna_residencia", "glosa_comuna_residencia"]
]
missing_capitulos.value_counts()

In [ ]:
egresos_diag_causa_df.drop(
    columns=["comuna_id", "comuna_residencia", "glosa_comuna_residencia"],
    inplace=True,
    errors="ignore",
)

egresos_diag_causa_df.head()

In [ ]:
# Ruta al archivo Excel en Google Drive
path_comunas = "/content/drive/MyDrive/TallerProyecto-Grupo2/datos/"

# Leer el archivo Excel
df_comunas = pd.read_csv(os.path.join(path_comunas, "ZONA_COMUNA.csv"), sep=";")
df_comunas["comuna"] = [
    "Peñalolén" if x == "Peñalolen" else x for x in df_comunas["comuna"]
]
# Mostrar el DataFrame<

In [ ]:
df_egresos = df_egresos.merge(
    df_comunas, left_on="glosa_comuna_residencia", right_on="comuna", how="left"
)
df_egresos.pop("comuna")
df_egresos.rename(
    columns={"zona": "servicio_salud", "zona_id": "id_servicio_salud"}, inplace=True
)
df_egresos = df_egresos[
    [
        "ano_egreso",
        "id_servicio_salud",
        "servicio_salud",
        "comuna_residencia",
        "glosa_comuna_residencia",
        "sexo",
        "grupo_edad",
        "prevision",
        "glosa_prevision",
        "diag1",
        "diag2",
        "dias_estad",
        "condicion_egreso",
    ]
]
df_egresos.columns = [col.lower() for col in df_egresos.columns]


# df_egresos.to_csv('/content/drive/MyDrive/TallerProyecto-Grupo2/datos/egresos.csv', index=False)

In [ ]:
diagnosticos = pd.read_csv(
    "/content/drive/MyDrive/TallerProyecto-Grupo2/datos/enfermedades.csv",
    sep=",",
    encoding="utf-8",
    header=None,
)
diagnosticos = diagnosticos.rename(
    columns={0: "diag1", 1: "subdiag", 2: "diag2", 3: "def1", 4: "def2", 5: "def"}
)

In [ ]:
df_egresos_def = pd.merge(
    df_egresos,
    diagnosticos[["diag1", "def"]].drop_duplicates(),
    left_on="diag1",
    right_on="diag1",
    how="left",
)
df_egresos_def = pd.merge(
    df_egresos_def,
    diagnosticos[["diag2", "def"]].drop_duplicates(),
    left_on="diag1",
    right_on="diag2",
    how="left",
)
df_egresos_def["def"] = [
    x if pd.notna(x) else y
    for x, y in zip(df_egresos_def["def_x"], df_egresos_def["def_y"])
]
df_egresos_def.pop("def_x")
df_egresos_def.pop("def_y")
df_egresos_def.pop("diag2_y")
df_egresos_def.rename(columns={"diag2_x": "diag2"}, inplace=True)
display(df_egresos_def.head())

In [ ]:
df_egresos_def_2 = df_egresos_def[
    (df_egresos_def["prevision"] == 1) & (df_egresos_def["def"].notna())
]

########analisis dias estadia servicio###########
df_egresos_def_dias_estad = (
    df_egresos_def_2[
        [
            "ano_egreso",
            "id_servicio_salud",
            "servicio_salud",
            "diag1",
            "def",
            "dias_estad",
        ]
    ]
    .groupby(["ano_egreso", "id_servicio_salud", "servicio_salud", "diag1", "def"])
    .mean("dias_estad")
    .reset_index()
)
#######analisis dias estadia enfermedad servicio

########analisis dias estadia###########
df_egresos_def_dias_estad = df_egresos_def_2.groupby(
    ["ano_egreso", "id_servicio_salud", "servicio_salud", "diag1", "def"],
    as_index=False,
).agg({"dias_estad": "mean"})
########analisis dias estadia###########

In [ ]:
###Estudio de enfermedades mas complejas estadia hospitalaria###

cantidad_casos = (
    df_egresos_def_dias_estad.groupby(
        ["ano_egreso", "id_servicio_salud", "servicio_salud", "def"]
    )
    .size()
    .reset_index(name="cantidad")
)
cantidad_casos["def"] = cantidad_casos["def"].str.strip()

df_egresos_def_dias_estad = df_egresos_def_dias_estad.apply(
    lambda x: x.strip() if type(x) == str else x
)

enfermedades_mayor_estadia = (
    df_egresos_def_dias_estad.groupby(
        ["ano_egreso", "id_servicio_salud", "servicio_salud", "def"], as_index=False
    )
    .agg({"dias_estad": "mean"})
    .sort_values(
        by=["ano_egreso", "id_servicio_salud", "servicio_salud", "dias_estad"],
        ascending=False,
    )
)

enfermedades_mayor_estadia_frec = pd.merge(
    enfermedades_mayor_estadia,
    cantidad_casos,
    left_on=["ano_egreso", "id_servicio_salud", "servicio_salud", "def"],
    right_on=["ano_egreso", "id_servicio_salud", "servicio_salud", "def"],
    how="left",
)
enfermedades_mayor_estadia_frec["dias_cama"] = (
    enfermedades_mayor_estadia_frec["dias_estad"]
    * enfermedades_mayor_estadia_frec["cantidad"]
)
enfermedades_mayor_estadia_frec = enfermedades_mayor_estadia_frec.sort_values(
    by=["ano_egreso", "id_servicio_salud", "cantidad"], ascending=False
)

In [ ]:
top_10_por_servicio = []
for csv_file in enfermedades_mayor_estadia_frec["ano_egreso"].unique():
    for servicio in enfermedades_mayor_estadia_frec["servicio_salud"].unique():
        top_10_por_servicio.append(
            enfermedades_mayor_estadia_frec[
                (enfermedades_mayor_estadia_frec["ano_egreso"] == csv_file)
                & (enfermedades_mayor_estadia_frec["servicio_salud"] == servicio)
            ].head(10)
        )

df_enfermedades = pd.concat(top_10_por_servicio)

df_enfermedades

In [ ]:
df_enfermedades_dias_estad = df_enfermedades[
    ["ano_egreso", "id_servicio_salud", "servicio_salud", "def", "dias_estad"]
]
lista_enfermedades = df_enfermedades["def"].to_list()
enfermedades = pd.DataFrame(lista_enfermedades, columns=["def"])

In [ ]:
df_egresos_def_dias_estad = df_egresos_def_dias_estad[
    ["ano_egreso", "id_servicio_salud", "servicio_salud", "def", "dias_estad"]
]
df_egresos_def_dias_estad = (
    df_egresos_def_dias_estad.groupby(
        ["ano_egreso", "id_servicio_salud", "servicio_salud", "def"], as_index=False
    )
    .mean("dias_estad")
    .reset_index(drop=True)
)
df_egresos_def_dias_estad

In [ ]:
servicio_salud = df_egresos_def_dias_estad[
    ["ano_egreso", "id_servicio_salud", "servicio_salud"]
].drop_duplicates()

servicio_enfermedades = pd.merge(servicio_salud, enfermedades, how="cross")

# servicio_enfermedades
# df_egresos_def_dias_estad
df_enfermedades_dias_estad = pd.merge(
    servicio_enfermedades,
    df_egresos_def_dias_estad,
    left_on=["ano_egreso", "id_servicio_salud", "servicio_salud", "def"],
    right_on=["ano_egreso", "id_servicio_salud", "servicio_salud", "def"],
    how="left",
)
df_enfermedades_dias_estad = df_enfermedades_dias_estad.fillna(0)
df_enfermedades_dias_estad

In [ ]:
diagnosticos_frecuentes_pivot_dias_estad = df_enfermedades_dias_estad.pivot_table(
    index=["ano_egreso", "id_servicio_salud", "servicio_salud"],
    columns="def",
    values=["dias_estad"],
    aggfunc="mean",  # Puedes usar 'mean', 'max', 'min', etc., dependiendo de tus necesidades
    fill_value=0,  # Esto reemplaza los NaN por 0
).reset_index()


# Desagrupar los índices de las columnas
diagnosticos_frecuentes_pivot_dias_estad.columns = [
    "_".join(col).strip() if isinstance(col, tuple) else col
    for col in diagnosticos_frecuentes_pivot_dias_estad.columns
]
diagnosticos_frecuentes_pivot_dias_estad.columns.values[0] = "ano_egreso"
diagnosticos_frecuentes_pivot_dias_estad.columns.values[1] = "id_servicio_salud"
diagnosticos_frecuentes_pivot_dias_estad.columns.values[2] = "servicio_salud"

display(diagnosticos_frecuentes_pivot_dias_estad)
diagnosticos_frecuentes_pivot_dias_estad.to_csv(
    "/content/drive/MyDrive/TallerProyecto-Grupo2/datos/dfegresosdiasestad.csv",
    index=False,
    sep=";",
    encoding="latin-1",
)

In [ ]:
df_egresos_def_cantidad = (
    df_egresos_def_2.groupby(
        ["ano_egreso", "id_servicio_salud", "servicio_salud", "def"]
    )
    .size()
    .reset_index(name="cantidad")
)

df_enfermedades_cantidad = pd.merge(
    df_egresos_def_cantidad,
    df_enfermedades_dias_estad,
    left_on=["ano_egreso", "id_servicio_salud", "servicio_salud", "def"],
    right_on=["ano_egreso", "id_servicio_salud", "servicio_salud", "def"],
    how="inner",
)

# df_enfermedades_cantidad.to_csv('/content/drive/MyDrive/TallerProyecto-Grupo2/datos/dfcantidaddecasos.csv', index=False,sep =';',encoding = 'latin-1')
diagnosticos_frecuentes_pivot_casos = df_enfermedades_cantidad.pivot_table(
    index=["ano_egreso", "id_servicio_salud", "servicio_salud"],
    columns="def",
    values=["cantidad"],
    aggfunc="sum",  # Puedes usar 'mean', 'max', 'min', etc., dependiendo de tus necesidades
    fill_value=0,  # Esto reemplaza los NaN por 0
).reset_index()


# Desagrupar los índices de las columnas
diagnosticos_frecuentes_pivot_casos.columns = [
    "_".join(col).strip() if isinstance(col, tuple) else col
    for col in diagnosticos_frecuentes_pivot_casos.columns
]
diagnosticos_frecuentes_pivot_casos.columns.values[0] = "ano_egreso"
diagnosticos_frecuentes_pivot_casos.columns.values[1] = "id_servicio_salud"
diagnosticos_frecuentes_pivot_casos.columns.values[2] = "servicio_salud"
diagnosticos_frecuentes_pivot_casos.columns = [
    x.replace("cantidad_", "")
    for x in diagnosticos_frecuentes_pivot_casos.columns.to_list()
]

# for col in diagnosticos_frecuentes_pivot_casos.columns.to_list()[3:]:
#   if len(diagnosticos_frecuentes_pivot_casos[diagnosticos_frecuentes_pivot_casos[col]==0][col])/len(diagnosticos_frecuentes_pivot_casos[col])>=0.7:
#     diagnosticos_frecuentes_pivot_casos.pop(col)

diagnosticos_frecuentes_pivot_casos.columns.values[3:] = [
    "casos_" + x for x in diagnosticos_frecuentes_pivot_casos.columns.to_list()[3:]
]


dfEgresosConsolidados = pd.merge(
    diagnosticos_frecuentes_pivot_dias_estad,
    diagnosticos_frecuentes_pivot_casos,
    left_on=["ano_egreso", "id_servicio_salud", "servicio_salud"],
    right_on=["ano_egreso", "id_servicio_salud", "servicio_salud"],
    how="left",
)
dfEgresosConsolidados.to_csv(
    "/content/drive/MyDrive/TallerProyecto-Grupo2/datos/dfEgresosConsolidados.csv",
    index=False,
    sep=";",
    encoding="latin-1",
)
dfEgresosConsolidados

In [ ]:
casen = pd.read_csv(
    "/content/drive/MyDrive/TallerProyecto-Grupo2/datos/casen_2017_2022_fonasa.csv",
    sep=",",
    encoding="utf-8",
)

total_fonasa = (
    casen.groupby(["capitulo", "nombre_capitulo"])
    .agg({"total_fonasa": "mean"})
    .reset_index()
)
total_fonasa = total_fonasa.rename(
    columns={"capitulo": "id_servicio_salud", "nombre_capitulo": "servicio_salud"}
)

casen = casen.rename(
    columns={"capitulo": "id_servicio_salud", "nombre_capitulo": "servicio_salud"}
)

casen = casen[
    ["id_servicio_salud", "servicio_salud"]
    + [x for x in casen.columns.to_list() if "porcentaje" in x]
]
casen = casen.groupby(["id_servicio_salud", "servicio_salud"]).mean().reset_index()
casen

In [ ]:
dfconsolidado = pd.merge(
    total_fonasa,
    diagnosticos_frecuentes_pivot_casos,
    left_on=["id_servicio_salud", "servicio_salud"],
    right_on=["id_servicio_salud", "servicio_salud"],
    how="left",
)
dfconsolidado[dfconsolidado.columns[4:]] = dfconsolidado[dfconsolidado.columns[4:]].div(
    dfconsolidado["total_fonasa"], axis=0
)
dfconsolidado = pd.merge(
    dfconsolidado,
    diagnosticos_frecuentes_pivot_dias_estad,
    left_on=["ano_egreso", "id_servicio_salud", "servicio_salud"],
    right_on=["ano_egreso", "id_servicio_salud", "servicio_salud"],
    how="left",
)
dfconsolidado.pop("total_fonasa")
dfconsolidado = dfconsolidado.fillna(0)
dfconsolidado.to_csv(
    "/content/drive/MyDrive/TallerProyecto-Grupo2/datos/dfconsolidado.csv",
    index=False,
    sep=";",
    encoding="latin-1",
)
dfconsolidado

In [ ]:
presupuesto = pd.read_csv(
    "/content/drive/MyDrive/TallerProyecto-Grupo2/datos/pagos_salud_chile_2018-2024.csv",
    sep=",",
    encoding="utf-8",
)
presupuesto = (
    presupuesto.groupby(["año", "capitulo", "nombre_capitulo", "nombre_subtitulo"])
    .agg({"devengo": "sum"})
    .reset_index()
)
presupuesto = presupuesto.rename(
    columns={
        "capitulo": "id_servicio_salud",
        "nombre_capitulo": "servicio_salud",
        "año": "ano_egreso",
    }
)
presupuesto = presupuesto.pivot_table(
    index=["ano_egreso", "id_servicio_salud", "servicio_salud"],
    columns="nombre_subtitulo",
    values="devengo",
    aggfunc="sum",
).reset_index()
presupuesto[presupuesto.columns[3:]] = presupuesto[presupuesto.columns[3:]] / 1000000

presupuesto

In [ ]:
dfconsolidado = pd.merge(
    presupuesto,
    dfconsolidado,
    left_on=["ano_egreso", "id_servicio_salud", "servicio_salud"],
    right_on=["ano_egreso", "id_servicio_salud", "servicio_salud"],
    how="left",
)
dfconsolidado = dfconsolidado[dfconsolidado["ano_egreso"] < 2024]
dfconsolidado = pd.merge(
    dfconsolidado,
    casen,
    left_on=["id_servicio_salud", "servicio_salud"],
    right_on=["id_servicio_salud", "servicio_salud"],
    how="left",
)
dfconsolidado

In [ ]:
dfconsolidado[dfconsolidado.columns[3:13]] = dfconsolidado[
    dfconsolidado.columns[3:13]
].div(dfconsolidado["total_fonasa"], axis=0)
dfconsolidado = dfconsolidado.fillna(0)
dfconsolidado.columns = dfconsolidado.columns.str.replace("\n", "").str.replace(
    " ", "_"
)
dfconsolidado

In [ ]:
###comienzo proceso de clusterizacion
import numpy as np
from sklearn.model_selection import train_test_split

dataset = dfconsolidado.copy()
dataset = dataset[dataset.columns[3:]]


from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3, random_state=42, n_init="auto")
kmeans = kmeans.fit(dataset)

print(kmeans.labels_)

dfconsolidado["clasificacion"] = kmeans.labels_

cluster = []
for x in dfconsolidado["clasificacion"]:
    if x == 0:
        cluster.append("medio")
    elif x == 1:
        cluster.append("bueno")
    else:
        cluster.append("malo")

dfconsolidado["clasificacion"] = cluster

dfconsolidado